In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from scipy.spatial import cKDTree
from shapely.geometry import Point
from preparacion_datos.src.preparacion_inter_puntos import prep
import glob
import tqdm as tqdm
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.geometry
import warnings
from shapely.geometry import shape
warnings.filterwarnings("ignore")

### PREDIAL

In [ ]:
casas=gpd.read_file(r'C:\Users\dlara\Naucalpan_new_casas.shp')
# terrenos=gpd.read_file(r'C:\Users\dlara\Downloads\naucalpancompleto\new_terreno1.shp')

In [ ]:
predial=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIAL (16022022).xlsx')
adeudo=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIALADEUDO (16022022).xlsx')
propietario=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PROPIETARIO (16022022).xlsx')

In [ ]:
base_pred= pd.read_csv(r'C:\Users\dlara\Downloads\Predial.csv')

In [ ]:
from geoloc2.Valor_Catastral.calculo_predial import *

In [ ]:
curts= gpd.read_file(r'C:\Users\dlara\Downloads\098 Naucalpan\NAUCALPAN_PREDIOS.shp')

In [ ]:
NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')

In [ ]:
def transform_df_to_gpd(df: pd.DataFrame,
                    lon_col: str = 'INTFIS_RS_FD_LON_1',
                    lat_col: str = 'INTFIS_RS_FD_LAT_1', crs= 'EPSG:4326'):
        '''Transforma base a partir de dos columnas de latitud y longitud'''
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(
                df[lon_col], df[lat_col]
            ),
            crs=crs,
        )
        return gdf

In [ ]:
NAUC['INTFIS_RS_FD_LAT']= NAUC['INTFIS_RS_FD_LAT_1'].combine_first(NAUC['INTFIS_RS_FD_LAT'])
NAUC['INTFIS_RS_FD_LON']= NAUC['INTFIS_RS_FD_LON_1'].combine_first(NAUC['INTFIS_RS_FD_LON'])

In [ ]:
NAUC=transform_df_to_gpd(NAUC, lon_col= 'INTFIS_RS_FD_LON',lat_col = 'INTFIS_RS_FD_LAT')

In [ ]:
PRED= NAUC.loc[NAUC['CLAVECATASTRAL'].notna()]

In [ ]:
Manzanas=gpd.read_file(r'C:\Users\dlara\Downloads\Full manzanas\Full manzanas\Manzana_Naucalpan.shp')

In [ ]:
Manzanas['geometry']=Manzanas.buffer(100)

In [ ]:
Manz=Manzanas.dissolve()

In [ ]:
NAUC=NAUC.to_crs(Manz.crs)
def new_func(NAUC):
    return gpd.sjoin(NAUC,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

rectificado_nauc=new_func(NAUC)

In [ ]:
curts=curts.to_crs(Manz.crs)
curts_rectificado=gpd.sjoin(curts,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

In [ ]:
casas=casas.to_crs(Manz.crs)
casas['ID']=casas.reset_index()['index']
casas_rectificado=gpd.sjoin(casas,Manz).drop(columns=[	'index_right',	'mun',	'zona',	'manz',	'cve_cat'])

## Cruces

In [ ]:
def line2poli(BASE):    
    linestring=BASE.loc[(BASE["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(BASE["geometry"].map(lambda y: isinstance(y,Polygon))==False)].dissolve()

    ARR=linestring.geometry.astype(str)
    print(ARR)
    if len(ARR)>0:
        # print(ARR)
        
        # print(l1)
        for i in range(len(ARR)):
            try:
                # print(ARR)
                l1=ARR.str.find('POLYGON')
                print(ARR)
                list_wkt = ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",'')
                # print(ARR,list_wkt)
                list_wkt= [list_wkt+str('))')]
                # print(list_wkt)
                list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
                multi=shapely.geometry.MultiPolygon(list_polygons)
                print(multi)
                # print('multi', multi)
                linestring['geometry'] = multi

            except:
                # print(ARR)
                l1=ARR.str.find('LINESTRING')
                # print(ARR.str[:l1])
                list_wkt = ARR.str.replace(ARR.str[:l1],'').str.replace("GEOMETRYCOLLECTION (LINESTRING ",'')
                list_wkt= [list_wkt+str('))')]
                # print(list_wkt)
                list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
                multi=shapely.geometry.MultiPolygon(list_polygons)
                # print('multi', multi)
                linestring['geometry'] = multi
                
            for col in dictt['cata']:
                try:
                    
                    BASE=BASE[~BASE[col].isin(linestring[col])]
                except:
                    pass
            
            BASE = pd.concat([BASE,linestring])

            return(BASE)

In [ ]:
linestring=shps_merged.loc[(shps_merged["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(shps_merged["geometry"].map(lambda y: isinstance(y,Polygon))==False)].dissolve()

ARR=linestring.geometry.astype(str)
print(ARR)

if len(ARR)>0:
    # print(ARR)
    
    # print(l1)
    for i in range(len(ARR)):
        try:
            # print(ARR)
            l1=ARR.str.find('POLYGON')
            print(ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",''))
            list_wkt = ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",'')
            # print(ARR,list_wkt)
            list_wkt= [list_wkt+str('))')]
            # print(list_wkt)
            list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
            multi=shapely.geometry.MultiPolygon(list_polygons)
            print(multi)
            # print('multi', multi)
            linestring['geometry'] = multi

        except:
            # print(ARR)
            l1=ARR.str.find('LINESTRING')
            # print(ARR.str[:l1])
            list_wkt = ARR.str.replace(ARR.str[:l1],'').str.replace("GEOMETRYCOLLECTION (LINESTRING ",'')
            list_wkt= [list_wkt+str('))')]
            # print(list_wkt)
            list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
            multi=shapely.geometry.MultiPolygon(list_polygons)
            # print('multi', multi)
            linestring['geometry'] = multi
            
        for col in dictt['cata']:
            try:
                
                shps_merged=shps_merged[~shps_merged[col].isin(linestring[col])]
            except:
                pass
        
        shps_merged = pd.concat([shps_merged,linestring])

In [ ]:
dictt= {'cata':['id_cat','cve_cat','CLAVECATAS','CLAVECATASTRAL_1','CLAVECATASTRAL_2']}
dictt['cata']

In [ ]:

path=r"C:\Users\dlara\Desktop\MUNICIPIOS\Insumos_municipios_en_proceso\*"
filenames= glob.glob(path)
for files in tqdm.tqdm(filenames):

    shapes={}
       
    i=0
    files=files.replace('\\',"/")
    path2=files.replace('\\',"/").replace('/*',"/")
    
    for file in glob.glob(path2 + '/*.shp'):
        file= file.replace('/',"\\")
        name=file.replace(path2.replace('/',"\\"),'').replace('.shp',"").replace('\\',"") 
        i=i+1
        Nname= name + str(i)
    #   
        shapes[Nname]=gpd.read_file(file)
        
        # print(Nname)
        # reemplazar id cat por clave catastral
        
        for col in dictt['cata']:
            shapes[Nname].rename(columns={col:'CLAVECATASTRAL'}, inplace=True)
        #Disolución de las geometrías a partir de clavecatastral 
        shapes[Nname]=shapes[Nname].dissolve(by='CLAVECATASTRAL').reset_index()
        
        if shapes[Nname].crs == 4326:
            pass
        else:
            shapes[Nname]=shapes[Nname].to_crs(4326)

    lista= list(shapes.keys())       
    for i in tqdm.tqdm(range(len(lista))):
        # if  (Nname.casefold() in i.casefold()):
        #     # print(Nname.casefold(), i.casefold())
        #     if Nname.casefold() == i.casefold():
                # print((Nname.casefold(), i.casefold()),)
                # continue
            # else:
        
        shps_merged= shapes[lista[i]]
        shps_merged.crs= 4326
        try:
            shps_merged=shapes[lista[i+1]].overlay(shps_merged, how='union',keep_geom_type=False)
            # print(shps_merged)
            shps_merged= line2poli(shps_merged)

        except:
            # print(shapes[lista[i]].head())
            shps_merged= line2poli(shapes[lista[i]])

            # print(name)
            # print(shps_merged)
            shps_merged.to_file(r'C:/Users/dlara/Desktop/MUNICIPIOS/' + Nname + "_cruce_shapes.shp")
            # else: 
            # print('error')  
            pass

In [ ]:
PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)
# 
PRED[['CLAVECATASTRAL', 'LONGITUD','LATITUD','CURT_1', 'curt','geometry','min_dist']].dropna()

# prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857).to_crs(4326)

PRED.drop_duplicates('geometry', inplace=True)
PRED.dropna(axis=1, inplace=True)
puntos=prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857)
puntos.dropna(axis=1, inplace=True)
rev_pred=gpd.read_file(r'C:\Users\dlara\Downloads\final\final\test_igecem_final.shp')

UNICOS=rev_pred.loc[rev_pred['CURT_1']==1]
nounicos=rev_pred.loc[rev_pred['CURT_1'].fillna(0)!=1]
nounicos=nounicos.to_crs(3857)
puntos=puntos.set_geometry('geometry')
rev_pred=rev_pred.set_geometry('geometry')
rev_pred.drop_duplicates('CLAVECATAS', keep='first', inplace=True)
# from preparacion_datos.src.preparacion_inter_puntos import ckdnearest

j_puntos=prep.ckdnearest(puntos, puntos[['geometry']])
cruce_check= gpd.sjoin(nounicos.to_crs(3857),j_puntos.loc[j_puntos['min_dist_2']>30][['geometry', 'LONGITUD_1', 'LATITUD_1','min_dist']], how='right')

cruce_check.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_reps.shp')
UNICOS.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_unicos.shp')

## Cruce_final

In [ ]:
def deduplicate(geo_data: np.ndarray # shape == (N, 4)
        ) -> np.ndarray:             # deduplicated data with origin order
    data = geo_data.reshape(-1, 2, 2)
    dt = f'f{data.itemsize}' # f4 or f8
    data = data.view([('x', dt), ('y', dt)]) 
    # eliminate differences
    ixs = np.argsort(data, -2, order=('x', 'y'))
    data_no_df = np.take_along_axis(data, ixs, axis=-2) # sorted by 'x' then by 'y'
    # get unique
    unique_sorted_data, uni_ixs = np.unique(data_no_df, True, axis=0)
    uni_ixs.sort() # inplace sort 1d-array
    data_deduplicated = geo_data[uni_ixs] # unique, originally ordered and shaped
    return data_deduplicated

In [ ]:
points= cruce_check.geometry
geoms = [shape(feat["geometry"]) for feat in points ]
list_arrays = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms ]
result = deduplicate(list_arrays)
final_result = [list(map(tuple, pair)) for pair in result.tolist()]
pnts = [Point(pair) for pair in final_result]
# cruce_check['geometry'] = pnts
geometria=gpd.GeoSeries(pnts)

In [4]:

def cruce_predial_adicionales():
    filenames= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\*.csv")

    for path in tqdm.tqdm(filenames):
        # filename= glob.glob(path)
        puntos=pd.read_csv(path)
        puntos= prep.transform_df_to_gpd(puntos, 'LONGITUD','LATITUD', 4326)
        rev_pred=gpd.read_file(path[:path.find('_puntos.csv')] + '.shp')
        rev_pred=rev_pred.to_crs(3857)
        UNICOS=rev_pred.loc[rev_pred['CURT_1']==1]
        nounicos=rev_pred.loc[rev_pred['CURT_1'].fillna(0)!=1]
        
        puntos=puntos.set_geometry('geometry')
        rev_pred=rev_pred.set_geometry('geometry')
        rev_pred.drop_duplicates('CLAVECATAS', keep='first', inplace=True)
        j_puntos=prep.ckdnearest(puntos.to_crs(3857), puntos[['geometry']].to_crs(3857), k=2)
        # print(nounicos, 'no unicos')
        # print(nounicos.crs, 'no unicos crs')
        # print(j_puntos, 'puntos')
        # print(j_puntos.crs,'puntos crs')
        # print(gpd.sjoin(nounicos,j_puntos.loc[j_puntos['min_dist_2']==0][['geometry', 'LONGITUD', 'LATITUD']]))
        cruce_check= gpd.sjoin(nounicos,j_puntos.loc[j_puntos['min_dist_2']==0][['geometry', 'LONGITUD', 'LATITUD']])
        # cruce_check=cruce_check.loc[cruce_check['CLAVECATASTRAL'].notna()]
        cruce_check=cruce_check.groupby('geometry', as_index=False,sort=False).first().reset_index(drop=True)
    #   cruce_check.loc[cruce_check['CLAVECATAS']=='0240127004000000'].plot()
        file_name='Salidas'
        name=path[path.rfind('\\')+1:path.rfind('_puntos')]
        path12= os.path.join(path[:path.rfind('MUNICIPIOS')],'MUNICIPIOS' ,file_name)
        # cruce_check.to_file(path12 + '\\' +name+  '_predial_final_curts_reps.shp')
        # UNICOS.to_file(path12 + '\\' +name+ '_predial_final_curts_unicos.shp')
        # final_pred= pd.concat([cruce_check, UNICOS])
        filename2= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\*.shp")
        for path2 in filename2:
            
            if name in path2:
                
                cruces_ads=gpd.read_file(path2)
                print(cruces_ads.crs)
                name2=path2[path2.rfind('\\'+name)+1:path2.rfind('_cruces_todos')]
            else:
                print('El archivo aun no se encuentra en la carpeta')
                break
            # if cruces_ads.crs== 4326:
            #     cruces_ads=cruces_ads.to_crs(3857)
            # elif cruces_ads.crs == 3857:
            #     continue           
            # print(cruces_ads.columns)
            ### el primero es fácil porque es directo, resultado: poli    
            primero=gpd.sjoin(UNICOS,cruces_ads)
            
            ## diferenciando
            temp1=cruces_ads.loc[~cruces_ads['CURT_f'].isin(primero['CURT_f'])]
            ##cruce
            drop=temp1.columns[(temp1.columns.str.contains('left|right'))]
            temp1.drop(columns=drop, inplace=True)
            drop=cruce_check.columns[(cruce_check.columns.str.contains('left|right'))]
            cruce_check.drop(columns=drop, inplace=True)
            segundo= gpd.sjoin(cruce_check,temp1)
       
            sobra_pred=cruce_check[~cruce_check.geometry.isin(segundo.geometry)]
            real_ads= temp1.loc[~temp1['CURT_f'].isin(segundo['CURT_f'])]
            sobra_pred['CAT_1']='FUERA'
            primero['CAT_1']='PREDIAL_UNICOS'
            segundo['CAT_1']= 'PREDIAL_REPETIDOS'
            pred_final= pd.concat([primero, segundo,sobra_pred])
            pred_final.dropna(how='all', axis=1, inplace=True)
            drop=pred_final.columns[(pred_final.columns.str.contains('0|TYPES|TYPE|ADDRESS|PLUS_CODE|_GEO_|Unnamed'))]
            pred_final.drop(columns=drop, inplace=True)
            
            pred_final.to_file(path12 + '\\' +name2+ 'base_final_predial_corregida.shp')
            real_ads['CAT_1']= 'ADICIONALES'
            base_final=pd.concat([pred_final, real_ads])
            base_final.to_file(path12 + '\\' +name2+ 'base_final_ads_pred.shp')
            return(base_final)

In [5]:
base_final=cruce_predial_adicionales()

  0%|          | 0/1 [00:00<?, ?it/s]

epsg:3857


In [ ]:
base_final.to_file(r'C:\Users\dlara\Desktop\MUNICIPIOS\Salidas\Ixtapan_terrenos_base_final_ads_pred.shp')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1076901742.py, line 1)

In [ ]:
filename2= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\*.shp")
for path2 in filename2:
    print(path2)

In [ ]:
name=path[path.rfind('\\')+1:path.rfind('_puntos')]
path12= os.path.join(path[:path.rfind('MUNICIPIOS')],'MUNICIPIOS' ,file_name)
path2[path2.rfind(path12 + '\\' +name):path2.rfind('_cruces_todos')]

In [ ]:
name

In [ ]:
filename2 + '\\' +name

In [ ]:
test0=pd.concat([m1,ADS_F,PRED_F, fin_casas], axis=0)
# .drop_duplicates('geometry')

In [ ]:
predial.loc[~predial['CLAVECATASTRAL'].isin(PRED['CLAVECATASTRAL'])]

### Distancias mínimas

In [ ]:
from shapely.geometry import Point, LineString
import geopandas as gpd
from geoloc2.preparacion_datos.src.preparacion_inter_puntos import ckdnearest
gdf=gpd.read_file("C:/Users/ASUS/Desktop/qgiszonas/points.shp")
gdf["LATLON"]=gdf["geometry"].x.astype("str")+","+gdf["geometry"].y.astype("str")
gdf


# usar primero sjoin_nearest y si hay mucho fallo usar la función ckdnearest

In [ ]:
casas=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\casas_t_especificios_1.shp")
casas=casas.to_crs(gdf.crs)
casas["LATLON"]=casas["geometry"].centroid.x.astype("str")+","+casas["geometry"].centroid.y.astype("str")
casas["geometry1"]=casas["geometry"]
casas["geometry"]=casas["geometry"].representative_point()
casas=casas.reset_index()
casas.rename(columns={"index":"FID"},inplace=True)
casas

In [ ]:
terreno=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\terreno_t_especificios_1.shp")
terreno=terreno.to_crs(gdf.crs)
terreno["LATLON"]=terreno["geometry"].centroid.x.astype("str")+","+terreno["geometry"].centroid.y.astype("str")
terreno["geometry1"]=terreno["geometry"]
terreno["geometry"]=terreno["geometry"].representative_point()
terreno

In [ ]:

# JFF_c = JFF_c.loc[JFF_c['min_dist'] < 10]
JFF_c

In [ ]:
JFF_terreno = ckdnearest(gdf.drop(columns='LATLON').to_crs(3857), terreno.to_crs(3857))
# JFF_terreno = JFF_terreno.loc[JFF_terreno['min_dist'] < 10]
JFF_terreno

In [ ]:
JFF_c["origen"]="Casas"
JFF_terreno["origen"]="Terreno"
concatenado=pd.concat([JFF_c,JFF_terreno],ignore_index=True)
concatenado.sort_values(by="min_dist",inplace=True)
concatenado=concatenado.drop_duplicates(subset=["CLAVECATAS"],keep="first")
concatenado

In [ ]:
concatenado[["FID","origen","CLAVECATAS","geometry"]].to_crs("3857").to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Puntos_cercanos.shp")


In [ ]:
a=concatenado[["FID","origen","CLAVECATAS","geometry1"]]
a=gpd.GeoDataFrame(a[["FID","origen","CLAVECATAS"]],geometry=a["geometry1"])
a.crs=6364
a=a.to_crs("3857")
a.to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Poligonos_cercanos.shp")

## Correcciones

In [ ]:
N= pd.read_csv('E:\Downloads\INTFIS_BASE_FINAL_NAUCALPAN_por_coma\INTFIS_BASE_FINAL_NAUCALPAN_por_coma.csv', encoding='utf-8-sig')

In [ ]:
PRED=N.loc[N['CLAVECATASTRAL_RECAUDACION'].notna()].dropna(how='all')

In [ ]:
correct_pred= PRED.loc[PRED['CLAVECATASTRAL'].astype(str).str.startswith('00')].groupby(['CLAVECATASTRAL','BDINTERNA_CONT_NOMBRE_COMPLETO']).first().dropna(axis=1,how='all').drop(columns='Unnamed: 0').reset_index()
# correct_pred['CLAVECATASTRAL']=correct_pred['CLAVECATASTRAL'].str[-12:].str.replace('000','', regex=False).str.zfill(10)

In [ ]:
correct_pred['CLAVECATASTRAL'].str[-16:]
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00000'), 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00000')]['CLAVECATASTRAL'].str[-11:].str.zfill(12).str.replace('000','', regex=False).str.zfill(10)

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('000'),'CLAVECATASTRAL']=correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('000')]['CLAVECATASTRAL'].str[-16:].str.replace('000','')

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00'),'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00')]['CLAVECATASTRAL'].str[:12].str.replace('00','').str.zfill(9)

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()<9, 'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()<9]['CLAVECATASTRAL'] + '00000000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==9, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==9]['CLAVECATASTRAL'] + '0000000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==10, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==10]['CLAVECATASTRAL'].str.zfill(10) + '000000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==12, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==12]['CLAVECATASTRAL'] + '0000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==13, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==13]['CLAVECATASTRAL'] + '000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==14, 'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==14]['CLAVECATASTRAL'] + '00'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==16, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==16]['CLAVECATASTRAL'].str[:-6].str.replace('098','98').str.ljust(15,fillchar='0').str.zfill(16)

In [ ]:
# PRED=PRED.loc[~PRED['CLAVECATASTRAL'].astype(str).str.startswith('00')]
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].str.replace('.0','', regex=False).str.zfill(16)

In [ ]:
# PRED=PRED.dropna(axis=1,how='all').drop(columns='Unnamed: 0')
ZPRED=pd.concat([PRED,correct_pred ], axis=0)
# ZPRED.CLAVECATASTRAL.value_counts()
ZPRED.loc[ZPRED['CLAVECATASTRAL'].isna(), 'CLAVECATASTRAL'] =ZPRED.loc[ZPRED['CLAVECATASTRAL'].isna()]['CLAVECATASTRAL_RECAUDACION'].str.replace('CAT-','').astype(str)

In [ ]:
duplicados=ZPRED.loc[ZPRED['CLAVECATASTRAL'].isin(ZPRED['CLAVECATASTRAL'][ZPRED['CLAVECATASTRAL'].duplicated()])].sort_values(by='CLAVECATASTRAL').groupby(['CLAVECATASTRAL','BDINTERNA_CONT_NOMBRE_COMPLETO']).first().reset_index()

In [ ]:
ZPRED=ZPRED.loc[~ZPRED['CLAVECATASTRAL'].isin(ZPRED['CLAVECATASTRAL'][ZPRED['CLAVECATASTRAL'].duplicated()])]
ZZPRED= pd.concat([ZPRED,duplicados], axis=0)

In [ ]:
ZZPRED['CLAVECATASTRAL_RECAUDACION'] = 'CAT-' + ZZPRED['CLAVECATASTRAL']

In [ ]:
NF=pd.read_csv(r'C:\Users\dlara\Downloads\drive-download-20230607T215848Z-001\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv',encoding='utf-8-sig')

In [ ]:
NFS=gpd.read_file(r'C:\Users\dlara\Downloads\drive-download-20230607T215848Z-001\BASE_MEJORADA_NAUCALPAN_URLS_v2.shp')

In [ ]:
PRED2=NF.loc[NF['CLAVECATASTRAL'].notna()]

In [ ]:
ZZPRED.loc[ZZPRED['CLAVECATASTRAL']=='09807751100E0904']

In [ ]:
PRED2['CLAVECATASTRAL']=PRED2['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)

In [ ]:
PRED2['CLAVECATASTRAL_RECAUDACION'] = 'CAT-' + PRED2['CLAVECATASTRAL']

In [ ]:

# denue=PRED2.columns[PRED2.columns.str.contains('DENUE')]
# PRED2.drop(columns=denue,inplace=True)
# drop1=PRED2.columns[(~PRED2.columns.isin(ZZPRED.columns)) & (PRED2.columns.str.contains('SAT|ESTATUS|NOMINA'))]
# PRED2.drop(columns=drop1,inplace=True)

drop2=PRED2.columns[(~PRED2.columns.isin(ZZPRED.columns)) & (PRED2.columns.str.contains('N_EXT|KEY|GRUPO_CONTROL|BDINTERNA|IMSS|CALLE|COLONIA|ADDRESS|PLUS_CODE|_GEO_|Unnamed'))]
PRED2.drop(columns=drop2,inplace=True)

In [ ]:
PRED2[['CLAVECATASTRAL','LATLON',  'CVEGEO',
       'Tipo Polígono', 'ORIGEN', 'TIPO DE COORDENADA', 'CURT_NOTAS', 'IMPORTE_FEB',
       'IMPORTE_MAR', 'ACTIVO', 'ACTIVIDAD_ECONOMICA', 'NUM_TRABAJADORES',
       'RANGO_TRABAJADORES', 'CLASIFICACION_TRABAJADORES', 'ACTIVO2',
       'CANTIDAD_ANIOS_ADEUDOS', 'ZONA_CATASTRAL', 'IMPORTE_FEB_X_ANIO',
       'CLASIFICACION_VALORCATASTRAL', 'USO', 'INTFIS_RS_FD_COOR_RECAUDACION',
       'INTFIS_RS_FD_ESTADO', 'INTFIS_RS_FD_PAIS', 'INTFIS_RS_FD_URLS',
       'CONSECUTIVO', 'RFC', 'RFC_CONTEOS', 'INTFIS_RS_FD_FULL_DOM', 'ID_CAT',
       'curt', 'DOMICILIO', 'NUM_EXT', 'NUM_INT', 'CP', 'CONST_DOM',
       'DOM_UNICOS', 'PERSONA', 'VALORCATASTRAL_2008', 'VALORCATASTRAL_2009',
       'VALORCATASTRAL_2010', 'VALORCATASTRAL_2011', 'VALORCATASTRAL_2012',
       'VALORCATASTRAL_2013', 'VALORCATASTRAL_2014', 'VALORCATASTRAL_2015',
       'PREDIAL_PROPIETARIO_EMAIL', 'PREDIAL_PROPIETARIO_CURP',
       'PREDIAL_PROPIETARIO_RFC', 'IMPORTETOTAL_ACUMULATIVO_AGUA',
       'CANTIDAD_NIS', 'NIS', 'CANTIDAD_CLAVES_CATASTRALES',
       'CLAVES_CATASTRALES', 'INTFIS_RS_FD_LATLON','curt','CURT','INTFIS_RS_FD_LON', 'INTFIS_RS_FD_LAT','INTFIS_RS_FD_LON_1', 'INTFIS_RS_FD_LAT_1','INTFIS_RS_FD_LAT_2', 'INTFIS_RS_FD_LON_2', '0', 'geometry']]

In [ ]:
ZZPRED.loc[~ZZPRED['CLAVECATASTRAL'].isin(PRED2['CLAVECATASTRAL'])]
# PRED2.loc[~PRED2['CLAVECATASTRAL'].isin(ZZPRED['CLAVECATASTRAL'])]